In [71]:
#Import libraries
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import transforms
import pandas as pd
from PIL import Image
import os
from tqdm import tqdm
import zipfile 
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score 
import pickle as pkl


In [3]:
device = torch.device('cuda:0')

In [72]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
#Upload dataset

df_training = pd.read_csv('drive/MyDrive/training.csv', sep="\t")
df_test = pd.read_csv('drive/MyDrive/trial.csv', sep="\t")

In [7]:
#CNN Model

class Binary_Classifier(nn.Module):
    def __init__(self):
        super(Binary_Classifier, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=10, kernel_size=3)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=3)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(58320, 1024)
        self.fc2 = nn.Linear(1024, 2)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x),2))
        x = x.view(x.shape[0],-1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

#hyper parameters

learning_rate = 0.01

# Initialize Model , Optimizer, Loss

model = Binary_Classifier().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0005)

In [8]:
# Due to the large input, the model is being trained in steps to avoid memory collapsing. It has been trained on the trial and training dataset.

#For the training set

steps = 10
step_training=int(df_training.shape[0]/steps)
step_test=int(df_test.shape[0]/steps)

for step in range(steps):
  input_training = []
  label_training = []
  img_zip = zipfile.ZipFile('drive/MyDrive/training.zip')
  img_zip.setpassword(b'*MaMiSemEval2022!')

  for i, row in tqdm(df_training.iloc[step_training*(step):step_training*(step+1)].iterrows()):
      try:
          filename = row['file_name']
          convert_tensor = transforms.Compose([
          transforms.Resize((224, 224)),
          transforms.ToTensor()
      ])
          img = Image.open(img_zip.open('TRAINING/' + filename)) 
          #img = img.resize((39, 39), Image.ANTIALIAS)
          tensor=convert_tensor(img)
          if tensor.shape[0] == 3:
              input_training.append(tensor)
              label_training.append(torch.tensor(row['misogynous'],dtype=torch.long))
      except:
          print('........................')
          continue

  #For the trial set
  
  img_zip = zipfile.ZipFile('drive/MyDrive/trial.zip')
  img_zip.setpassword(b'*MaMiSemEval2022!')
  for i, row in tqdm(df_test.iloc[step_test*step:step_test*(step+1)].iterrows()):
      try:
          filename = row['file_name']
          convert_tensor = transforms.Compose([
          transforms.Resize((224, 224)),
          transforms.ToTensor()
      ])
          img = Image.open(img_zip.open('Users/fersiniel/Desktop/MAMI - TO LABEL/TRIAL DATASET/' + filename)) 
          #img = img.resize((39, 39), Image.ANTIALIAS)
          tensor=convert_tensor(img)
          if tensor.shape[0] == 3:
              input_training.append(tensor)
              label_training.append(torch.tensor(row['misogynous'],dtype=torch.long))
      except Exception as E:
          
          print(E)
          continue

  train_tensor = torch.stack(input_training,0).to(device=device) 
  label_tensor = torch.stack(label_training,0).to(device=device)
  del input_training, label_training
  input, input_t, label, label_t = train_test_split(train_tensor, label_tensor, test_size=0.2)
  del train_tensor, label_tensor

  
  class dataset(Dataset):
    def __init__(self,x,y):
      self.x = x
      self.y = y
      self.length = self.x.shape[0]

    def __getitem__(self,idx):
      return self.x[idx],self.y[idx]
    def __len__(self):
      return self.length
  trainset = dataset(input, label)
 
  #DataLoader
  
  trainloader = DataLoader(trainset,batch_size=32,shuffle=True)


  #forward loop
 
  losses = []
  accur = []
  epochs = 5
  for i in tqdm(range(epochs)):
      for j,(x_train,y_train) in enumerate(trainloader):

          #calculate output
         
          output = model(x_train)

          #calculate loss
          #loss = loss_fn(output,y_train.reshape(-1,1))
          
          loss = loss_fn(output.reshape(-1,2),y_train)


          #backprop
          
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
          
          #accuracy
          
          if j%10 == 0:
              with torch.no_grad():
                  losses.append(loss)
                  acc =(label == torch.argmax(model(input),dim=1)).sum()/label.size()[0]
                  acc_t = (label_t == torch.argmax(model(input_t),dim=1)).sum()/label_t.size()[0]
                  accur.append(acc)
                  print(f"\nstep: {step}, epoch {i}, loss : {loss}, training accuracy : {acc}, test accuracy : {acc_t}")

1000it [00:11, 90.13it/s]
10it [00:03,  2.56it/s]
  0%|          | 0/5 [00:00<?, ?it/s]


step: 0, epoch 0, loss : 0.6874208450317383, training accuracy : 0.5284653306007385, test accuracy : 0.49504950642585754

step: 0, epoch 0, loss : 0.718193531036377, training accuracy : 0.551980197429657, test accuracy : 0.5990098714828491


 20%|██        | 1/5 [00:02<00:08,  2.07s/it]


step: 0, epoch 0, loss : 0.6663088202476501, training accuracy : 0.608910858631134, test accuracy : 0.5841584205627441

step: 0, epoch 1, loss : 0.6272650361061096, training accuracy : 0.6670792102813721, test accuracy : 0.6831682920455933

step: 0, epoch 1, loss : 0.5917803049087524, training accuracy : 0.6683168411254883, test accuracy : 0.6485148668289185


 40%|████      | 2/5 [00:03<00:05,  1.93s/it]

...


step: 9, epoch 2, loss : 0.15817803144454956, training accuracy : 0.9888476133346558, test accuracy : 0.5594059228897095

step: 9, epoch 3, loss : 0.07395579665899277, training accuracy : 0.9950433969497681, test accuracy : 0.5297029614448547

step: 9, epoch 3, loss : 0.009290333837270737, training accuracy : 0.9925650954246521, test accuracy : 0.5297029614448547


 80%|████████  | 4/5 [00:07<00:01,  1.88s/it]


step: 9, epoch 3, loss : 0.037056103348731995, training accuracy : 0.9962825775146484, test accuracy : 0.5247524380683899

step: 9, epoch 4, loss : 0.012129448354244232, training accuracy : 0.9950433969497681, test accuracy : 0.5297029614448547

step: 9, epoch 4, loss : 0.1153082624077797, training accuracy : 0.9950433969497681, test accuracy : 0.5297029614448547


100%|██████████| 5/5 [00:09<00:00,  1.86s/it]


step: 9, epoch 4, loss : 0.013944339007139206, training accuracy : 0.997521698474884, test accuracy : 0.5049504637718201


# Test

In [35]:
#Upoad and read test data
img_zip = zipfile.ZipFile('drive/MyDrive/test.zip')
img_zip.setpassword(b'*MaMiSemEval2022!')
df_test = pd.read_csv('drive/MyDrive/test_labels.txt',delimiter="\t", header=None)

In [46]:
df_test=df_test[0].str.split('        ',expand=True)[[0,1]].rename(columns={0:'file_name',1:'misogynous'})

In [51]:
df_test.misogynous=df_test.misogynous.astype('int32')

In [53]:
#Pre-processing test data
input = []
label = []
img_zip = zipfile.ZipFile('drive/MyDrive/test.zip')
img_zip.setpassword(b'*MaMiSemEval2022!')

for i, row in tqdm(df_test.iterrows()):
    try:
        filename = row['file_name']
        convert_tensor = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])

         
        
        img = Image.open(img_zip.open('test/' + filename)) 
        
        tensor=convert_tensor(img)
        if tensor.shape[0] == 3:
            input.append(tensor)
            label.append(torch.tensor(row['misogynous'],dtype=torch.long))
    except Exception as e:
        print(e)
        continue

1000it [03:52,  4.30it/s]


In [26]:
#del x_train, y_train, trainset, trainloader, tensor, output, label_t, input_t

In [54]:
test_tensor = torch.stack(input,0).to(device=device) 
test_label_tensor = torch.stack(label,0).to(device=device)

In [70]:
output = torch.argmax(model(test_tensor),dim=1)
output

tensor([1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1,
        1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0,
        0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0,
        1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1,
        1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1,
        1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0,
        1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0,
        0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0,
        1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0,
        0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1,
        1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0,
        1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0,

In [55]:
#pred=output

In [61]:
output.shape

torch.Size([1000])

In [63]:
test_label_tensor.cpu().detach().numpy()

array([0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1,
       0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0,
       1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1,
       1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1,
       0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,

In [67]:

#np_arr = torch_tensor.cpu().detach().numpy()

f1_score = f1_score(test_label_tensor.cpu().detach().numpy(), output.cpu().detach().numpy())
print(f1_score)

0.6232394366197183


In [74]:
#Create a pickle file of the predicted labels of the cnn to be used for the fusion

arrayInput = output #Trial input
fileName = "drive/MyDrive/cnn_output.pkl"
fileObject = open(fileName, 'wb')
pkl.dump(arrayInput, fileObject)
fileObject.close()